In [64]:
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

In [36]:

# load the matching document contents
df_text = pd.read_csv("data/trackGBV_xls_match.csv")
print(len(df_text))

809


In [38]:
df_labels = pd.read_csv("data/trackGBV_labels.csv")
print(len(df_labels))

809


In [ ]:
# Get placeholder for start of actual text
import re 
  
df_text['contents']= df_text['contents'].str.replace("\nJUDGMENT", "\nDF_CUTOFF_MARK", case = True) 
df_text['contents']= df_text['contents'].str.replace("\nSENTENCE", "\nDF_CUTOFF_MARK", case = True)
df_text['contents']= df_text['contents'].str.replace("\nRULING", "\nDF_CUTOFF_MARK", case = True)

In [31]:
# This takes a long time (> 30 minutes)
text2 = [re.sub(r'((.|\n)*)\nDF_CUTOFF_MARK','', i) for i in text] # placeholder for religon MAKES IT WORSE

In [34]:
# save as csv
import csv

with open('cleaned_text.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(text2)

In [135]:
data = text
labels = list(df_labels['Discrimination_Label'])

In [142]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=1)

In [153]:
from sklearn.feature_extraction.text import *
cv = CountVectorizer(strip_accents='ascii', 
                     lowercase=True, 
                     token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b',
                     stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [149]:
print('The size of the vocabulary is ', vect.shape[1], sep = '')
print('The average number of non-zero features per example is ', round(vect.nnz/vect.shape[0],4), sep = '')
print('The fraction of the entries in the matrix that are non-zero is ', vect.nnz, '/', vect.shape[0] * vect.shape[1], '(', round(vect.nnz / (vect.shape[0] * vect.shape[1]),4), ')',  sep = '')
print('The 0th and last feature strings are "', vectorizer.get_feature_names()[0], '" and "', vectorizer.get_feature_names()[-1], '"', sep = '')

bt_vectorizer = CountVectorizer(analyzer = 'char', ngram_range=(2, 3))
bt_vect = bt_vectorizer.fit_transform(X_train)
print('The size of the vocabulary with bigram and trigrams is ', len(bt_vectorizer.vocabulary_), sep = '')

The size of the vocabulary is 16210
The average number of non-zero features per example is 449.1422
The fraction of the entries in the matrix that are non-zero is 290595/10487870(0.0277)
The 0th and last feature strings are "00" and "½cm"
The size of the vocabulary with bigram and trigrams is 20829


In [150]:
# check training data
word_freq_df = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
top_words_df = pd.DataFrame(word_freq_df.sum()).sort_values(0, ascending=False)
top_words_df.head()
top_words_df.head(-5)

,0
court,8888
years,7924
sentence,7134
accused,5082
state,4780
...,...
lcj,1
laxity,1
lawgivers,1
lawfulness,1


In [156]:
# train and predict
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
naive_bayes =MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)

In [157]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy: ', round(accuracy_score(y_test, predictions),3))
print('Precision: ', round(precision_score(y_test, predictions),3))
print('Recall: ', round(recall_score(y_test, predictions),3))

Accuracy:  0.522
Precision:  0.571
Recall:  0.596


In [105]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C = 1.0)

In [106]:
clf.fit(X_train_cv,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [107]:
predictions_svc = clf.predict(X_test_cv)

In [108]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy score: ', accuracy_score(y_test, predictions_svc))
print('Precision score: ', precision_score(y_test, predictions_svc))
print('Recall score: ', recall_score(y_test, predictions_svc))

Accuracy score:  0.4567901234567901
Precision score:  0.5227272727272727
Recall score:  0.5


In [94]:
LR = LogisticRegression(random_state=0, solver='liblinear',multi_class='ovr').fit(X_train_cv, y_train)
predictions_LR = LR.predict(X_test_cv)
print('Accuracy score: ', accuracy_score(y_test, predictions_LR))
print('Precision score: ', precision_score(y_test, predictions_LR))
print('Recall score: ', recall_score(y_test, predictions_LR))

Accuracy score:  0.4382716049382716
Precision score:  0.5054945054945055
Recall score:  0.5


In [109]:
df_text['contents']

0      Home | Databases | WorldLII | Search | Feedbac...
1      Home | Databases | WorldLII | Search | Feedbac...
2         State v Lagivere - Sentence [2017] FJHC 386...
3         State v Goundar - Sentence [2018] FJHC 438;...
4      Home | Databases | WorldLII | Search | Feedbac...
                             ...                        
804    Home | Databases | WorldLII | Search | Feedbac...
805       State v Khelawan [2016] FJMC 41; Criminal C...
806       Bulivou v State [2014] FJCA 215; AAU78.2010...
807       State v Lal [2015] FJMC 58; Criminal Case 1...
808       Ram v State [2015] FJSC 26; CAV12.2015 (23 ...
Name: contents, Length: 809, dtype: object